In [435]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

import numpy as np
from matplotlib import pyplot as plt
import math
import statistics as stat

import sqlalchemy
from sqlalchemy import create_engine
import psycopg2
from config import db_password

import datetime
from datetime import date
from datetime import time

import re

## Read Kaggle data from matches.csv

Import the table `matches.csv` from Kaggle data on major league soccer (MLS).

In [362]:
# Read .csv from FinalProject/Resources
file_dir = "../Resources/matches.csv"    

# Create a pandas DataFrame 
game_df = pd.read_csv(file_dir, low_memory=False, skipinitialspace = True)

# Show all columns
pd.set_option("display.max_columns", None)

game_df

,id,home,away,date,year,time (utc),attendance,venue,league,part_of_competition,game_status,shootout,home_score,away_score,referee,home_goal_minutes,home_goal_scorers,away_goal_minutes,away_goal_scorers,home_starting_1_num,home_starting_1,home_starting_2_num,home_starting_2,home_starting_3_num,home_starting_3,home_starting_4_num,home_starting_4,home_starting_5_num,home_starting_5,home_starting_6_num,home_starting_6,home_starting_7_num,home_starting_7,home_starting_8_num,home_starting_8,home_starting_9_num,home_starting_9,home_starting_10_num,home_starting_10,home_starting_11_num,home_starting_11,home_bench_1_num,home_bench_1,home_bench_1_minute,home_bench_2_num,home_bench_2,home_bench_2_minute,home_bench_3_num,home_bench_3,home_bench_3_minute,home_bench_4_num,home_bench_4,home_bench_4_minute,home_bench_5_num,home_bench_5,home_bench_5_minute,home_bench_6_num,home_bench_6,home_bench_6_minute,home_bench_7_num,home_bench_7,home_bench_7_minute,home_bench_8_num,home_bench_8,home_bench_8_minute,home_bench_9_num,home_bench_9,home_bench_9_minute,away_starting_1_num,away_starting_1,away_starting_2_num,away_starting_2,away_starting_3_num,away_starting_3,away_starting_4_num,away_starting_4,away_starting_5_num,away_starting_5,away_starting_6_num,away_starting_6,away_starting_7_num,away_starting_7,away_starting_8_num,away_starting_8,away_starting_9_num,away_starting_9,away_starting_10_num,away_starting_10,away_starting_11_num,away_starting_11,away_bench_1_num,away_bench_1,away_bench_1_minute,away_bench_2_num,away_bench_2,away_bench_2_minute,away_bench_3_num,away_bench_3,away_bench_3_minute,away_bench_4_num,away_bench_4,away_bench_4_minute,away_bench_5_num,away_bench_5,away_bench_5_minute,away_bench_6_num,away_bench_6,away_bench_6_minute,away_bench_7_num,away_bench_7,away_bench_7_minute,away_bench_8_num,away_bench_8,away_bench_8_minute,away_bench_9_num,away_bench_9,away_bench_9_minute,home_bench_10_num,home_bench_10,home_bench_10_minute,home_starting_12_num,home_starting_12,home_starting_13_num,home_starting_13,home_starting_14_num,home_starting_14,away_starting_12_num,away_starting_12,away_starting_13_num,away_starting_13,away_starting_14_num,away_starting_14,away_starting_15_num,away_starting_15,home_starting_15_num,home_starting_15,home_starting_16_num,home_starting_16,home_starting_17_num,home_starting_17,home_starting_18_num,home_starting_18,home_starting_19_num,home_starting_19,home_starting_20_num,home_starting_20,home_starting_21_num,home_starting_21,home_starting_22_num,home_starting_22,home_bench_11_num,home_bench_11,home_bench_11_minute,home_bench_12_num,home_bench_12,home_bench_12_minute,home_bench_13_num,home_bench_13,home_bench_13_minute,home_bench_14_num,home_bench_14,home_bench_14_minute,away_starting_16_num,away_starting_16,away_starting_17_num,away_starting_17,away_starting_18_num,away_starting_18,away_starting_19_num,away_starting_19,away_starting_20_num,away_starting_20,away_starting_21_num,away_starting_21,away_bench_10_num,away_bench_10,away_bench_10_minute,away_bench_11_num,away_bench_11,away_bench_11_minute,away_bench_12_num,away_bench_12,away_bench_12_minute,away_starting_22_num,away_starting_22,away_bench_13_num,away_bench_13,away_bench_13_minute,home_possessionPct,away_possessionPct,home_shotsSummary,away_shotsSummary,home_foulsCommitted,away_foulsCommitted,home_yellowCards,away_yellowCards,home_redCards,away_redCards,home_offsides,away_offsides,home_wonCorners,away_wonCorners,home_saves,away_saves,away_bench_14_num,away_bench_14,away_bench_14_minute,home_formation,away_formation
0,NaN,New England,San Jose,7/31/1996,1996,NaN,"12,871",Foxboro Stadium,1996 MLS,Regular Season,FT,NaN,2,0,Ruben Rodhas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [363]:
results = game_df['home_score'] - game_df['away_score']
game_df['result'] = results
game_df.head(2)

,id,home,away,date,year,time (utc),attendance,venue,league,part_of_competition,game_status,shootout,home_score,away_score,referee,home_goal_minutes,home_goal_scorers,away_goal_minutes,away_goal_scorers,home_starting_1_num,home_starting_1,home_starting_2_num,home_starting_2,home_starting_3_num,home_starting_3,home_starting_4_num,home_starting_4,home_starting_5_num,home_starting_5,home_starting_6_num,home_starting_6,home_starting_7_num,home_starting_7,home_starting_8_num,home_starting_8,home_starting_9_num,home_starting_9,home_starting_10_num,home_starting_10,home_starting_11_num,home_starting_11,home_bench_1_num,home_bench_1,home_bench_1_minute,home_bench_2_num,home_bench_2,home_bench_2_minute,home_bench_3_num,home_bench_3,home_bench_3_minute,home_bench_4_num,home_bench_4,home_bench_4_minute,home_bench_5_num,home_bench_5,home_bench_5_minute,home_bench_6_num,home_bench_6,home_bench_6_minute,home_bench_7_num,home_bench_7,home_bench_7_minute,home_bench_8_num,home_bench_8,home_bench_8_minute,home_bench_9_num,home_bench_9,home_bench_9_minute,away_starting_1_num,away_starting_1,away_starting_2_num,away_starting_2,away_starting_3_num,away_starting_3,away_starting_4_num,away_starting_4,away_starting_5_num,away_starting_5,away_starting_6_num,away_starting_6,away_starting_7_num,away_starting_7,away_starting_8_num,away_starting_8,away_starting_9_num,away_starting_9,away_starting_10_num,away_starting_10,away_starting_11_num,away_starting_11,away_bench_1_num,away_bench_1,away_bench_1_minute,away_bench_2_num,away_bench_2,away_bench_2_minute,away_bench_3_num,away_bench_3,away_bench_3_minute,away_bench_4_num,away_bench_4,away_bench_4_minute,away_bench_5_num,away_bench_5,away_bench_5_minute,away_bench_6_num,away_bench_6,away_bench_6_minute,away_bench_7_num,away_bench_7,away_bench_7_minute,away_bench_8_num,away_bench_8,away_bench_8_minute,away_bench_9_num,away_bench_9,away_bench_9_minute,home_bench_10_num,home_bench_10,home_bench_10_minute,home_starting_12_num,home_starting_12,home_starting_13_num,home_starting_13,home_starting_14_num,home_starting_14,away_starting_12_num,away_starting_12,away_starting_13_num,away_starting_13,away_starting_14_num,away_starting_14,away_starting_15_num,away_starting_15,home_starting_15_num,home_starting_15,home_starting_16_num,home_starting_16,home_starting_17_num,home_starting_17,home_starting_18_num,home_starting_18,home_starting_19_num,home_starting_19,home_starting_20_num,home_starting_20,home_starting_21_num,home_starting_21,home_starting_22_num,home_starting_22,home_bench_11_num,home_bench_11,home_bench_11_minute,home_bench_12_num,home_bench_12,home_bench_12_minute,home_bench_13_num,home_bench_13,home_bench_13_minute,home_bench_14_num,home_bench_14,home_bench_14_minute,away_starting_16_num,away_starting_16,away_starting_17_num,away_starting_17,away_starting_18_num,away_starting_18,away_starting_19_num,away_starting_19,away_starting_20_num,away_starting_20,away_starting_21_num,away_starting_21,away_bench_10_num,away_bench_10,away_bench_10_minute,away_bench_11_num,away_bench_11,away_bench_11_minute,away_bench_12_num,away_bench_12,away_bench_12_minute,away_starting_22_num,away_starting_22,away_bench_13_num,away_bench_13,away_bench_13_minute,home_possessionPct,away_possessionPct,home_shotsSummary,away_shotsSummary,home_foulsCommitted,away_foulsCommitted,home_yellowCards,away_yellowCards,home_redCards,away_redCards,home_offsides,away_offsides,home_wonCorners,away_wonCorners,home_saves,away_saves,away_bench_14_num,away_bench_14,away_bench_14_minute,home_formation,away_formation,result
0,NaN,New England,San Jose,7/31/1996,1996,NaN,"12,871",Foxboro Stadium,1996 MLS,Regular Season,FT,NaN,2,0,Ruben Rodhas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [364]:
results

0       2
1       0
2      -1
3       1
4       2
       ..
6693   -4
6694    2
6695    0
6696    0
6697    0
Length: 6698, dtype: int64

In [365]:
# Create a new column "Outcome"
outcome = []
for i in range(len(results)):
    if results[i] > 0:
        outcome.append('win')
    elif results[i] < 0:
        outcome.append('loss')
    else:
        outcome.append('tie')
    
game_df['Outcome'] = outcome
game_df.head(2)

,id,home,away,date,year,time (utc),attendance,venue,league,part_of_competition,game_status,shootout,home_score,away_score,referee,home_goal_minutes,home_goal_scorers,away_goal_minutes,away_goal_scorers,home_starting_1_num,home_starting_1,home_starting_2_num,home_starting_2,home_starting_3_num,home_starting_3,home_starting_4_num,home_starting_4,home_starting_5_num,home_starting_5,home_starting_6_num,home_starting_6,home_starting_7_num,home_starting_7,home_starting_8_num,home_starting_8,home_starting_9_num,home_starting_9,home_starting_10_num,home_starting_10,home_starting_11_num,home_starting_11,home_bench_1_num,home_bench_1,home_bench_1_minute,home_bench_2_num,home_bench_2,home_bench_2_minute,home_bench_3_num,home_bench_3,home_bench_3_minute,home_bench_4_num,home_bench_4,home_bench_4_minute,home_bench_5_num,home_bench_5,home_bench_5_minute,home_bench_6_num,home_bench_6,home_bench_6_minute,home_bench_7_num,home_bench_7,home_bench_7_minute,home_bench_8_num,home_bench_8,home_bench_8_minute,home_bench_9_num,home_bench_9,home_bench_9_minute,away_starting_1_num,away_starting_1,away_starting_2_num,away_starting_2,away_starting_3_num,away_starting_3,away_starting_4_num,away_starting_4,away_starting_5_num,away_starting_5,away_starting_6_num,away_starting_6,away_starting_7_num,away_starting_7,away_starting_8_num,away_starting_8,away_starting_9_num,away_starting_9,away_starting_10_num,away_starting_10,away_starting_11_num,away_starting_11,away_bench_1_num,away_bench_1,away_bench_1_minute,away_bench_2_num,away_bench_2,away_bench_2_minute,away_bench_3_num,away_bench_3,away_bench_3_minute,away_bench_4_num,away_bench_4,away_bench_4_minute,away_bench_5_num,away_bench_5,away_bench_5_minute,away_bench_6_num,away_bench_6,away_bench_6_minute,away_bench_7_num,away_bench_7,away_bench_7_minute,away_bench_8_num,away_bench_8,away_bench_8_minute,away_bench_9_num,away_bench_9,away_bench_9_minute,home_bench_10_num,home_bench_10,home_bench_10_minute,home_starting_12_num,home_starting_12,home_starting_13_num,home_starting_13,home_starting_14_num,home_starting_14,away_starting_12_num,away_starting_12,away_starting_13_num,away_starting_13,away_starting_14_num,away_starting_14,away_starting_15_num,away_starting_15,home_starting_15_num,home_starting_15,home_starting_16_num,home_starting_16,home_starting_17_num,home_starting_17,home_starting_18_num,home_starting_18,home_starting_19_num,home_starting_19,home_starting_20_num,home_starting_20,home_starting_21_num,home_starting_21,home_starting_22_num,home_starting_22,home_bench_11_num,home_bench_11,home_bench_11_minute,home_bench_12_num,home_bench_12,home_bench_12_minute,home_bench_13_num,home_bench_13,home_bench_13_minute,home_bench_14_num,home_bench_14,home_bench_14_minute,away_starting_16_num,away_starting_16,away_starting_17_num,away_starting_17,away_starting_18_num,away_starting_18,away_starting_19_num,away_starting_19,away_starting_20_num,away_starting_20,away_starting_21_num,away_starting_21,away_bench_10_num,away_bench_10,away_bench_10_minute,away_bench_11_num,away_bench_11,away_bench_11_minute,away_bench_12_num,away_bench_12,away_bench_12_minute,away_starting_22_num,away_starting_22,away_bench_13_num,away_bench_13,away_bench_13_minute,home_possessionPct,away_possessionPct,home_shotsSummary,away_shotsSummary,home_foulsCommitted,away_foulsCommitted,home_yellowCards,away_yellowCards,home_redCards,away_redCards,home_offsides,away_offsides,home_wonCorners,away_wonCorners,home_saves,away_saves,away_bench_14_num,away_bench_14,away_bench_14_minute,home_formation,away_formation,result,Outcome
0,NaN,New England,San Jose,7/31/1996,1996,NaN,"12,871",Foxboro Stadium,1996 MLS,Regular Season,FT,NaN,2,0,Ruben Rodhas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [366]:
# Keep only those columns that information can be known before a game starts.
game_df = game_df[['home', 'away', 'date', 'year', 'time (utc)', 'attendance', 'venue', 'Outcome']]
game_df

,home,away,date,year,time (utc),attendance,venue,Outcome
0,New England,San Jose,7/31/1996,1996,NaN,"12,871",Foxboro Stadium,win
1,Dallas,Colorado,6/15/1996,1996,NaN,"9,704",Cotton Bowl,tie
2,Colorado,D.C. United,8/29/1996,1996,NaN,"6,368",Mile High Stadium,loss
3,LA Galaxy,New England,8/8/1996,1996,NaN,"10,251",Rose Bowl,win
4,New England,D.C. United,7/20/1996,1996,NaN,"18,347",Foxboro Stadium,win
...,...,...,...,...,...,...,...,...
6693,Vancouver Whitecaps,Real Salt Lake,"Wednesday, July 7",2021,2:00,"12,009","Rio Tinto Stadium, Sandy, UT",loss
6694,LA Galaxy,FC Dallas,"Wednesday, July 7",2021,2:30,"16,453","Dignity Health Sports Park, Carson",win
6695,Nashville SC,Atlanta United FC,"Thursday, July 8",2021,0:30,"22,913","Nissan Stadium, Nashville, Nashville, TN",tie
6696,New York Red Bulls,Philadelphia Union,"Thursday, July 8",2021,0:30,"10,013","Red Bull Arena, Harrison",tie


In [367]:
# Replace the ',' from attendance values.

game_df['attendance'] = game_df['attendance'].str.replace(",", "").astype("float")

game_df.head(2)

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,home,away,date,year,time (utc),attendance,venue,Outcome
0,New England,San Jose,7/31/1996,1996,NaN,12871.0,Foxboro Stadium,win
1,Dallas,Colorado,6/15/1996,1996,NaN,9704.0,Cotton Bowl,tie


In [368]:
# Check null values
game_df.isnull().sum()

home             0
away             0
date             0
year             0
time (utc)     972
attendance    2355
venue          467
Outcome          0
dtype: int64

In [369]:
# Check for any duplicate venue
game_df['venue'].unique()

array(['Foxboro Stadium', 'Cotton Bowl', 'Mile High Stadium', 'Rose Bowl',
       'Robert F. Kennedy Memorial Stadium', "Houlihan's Stadium",
       'Giants Stadium', 'Arrowhead Stadium', 'Ohio Stadium',
       'CEFCU Stadium', 'Foxboro Stadium (Neutral Site)', 'Soldier Field',
       'Lockhart Stadium', 'Rose Bowl (Neutral Site)',
       'Raymond James Stadium', 'MAPFRE Stadium',
       'Robert F. Kennedy Memorial Stadium (Neutral Site)', nan,
       'AT THE ORANGE BOWL', 'Gillette Stadium, Foxborough',
       'Dignity Health Sports Park, Los Angeles',
       'Spartan Stadium, San Jose', 'MAPFRE Stadium, Columbus',
       'RFK Stadium, Washington, D.C.', 'Arrowhead Stadium, Kansas City',
       'Cardinal Stadium, Chicago',
       'Sports Authority Field at Mile High, Denver',
       'Giants Stadium, East Rutherford', 'Dragon Stadium, Dallas',
       'Soldier Field, Chicago', 'Cotton Bowl, Dallas',
       'Rice-Eccles Stadium, Salt Lake City',
       'Frisco Sports & Entertainment Cent

In [370]:
# No. of venues before removing duplicate venue names
game_df['venue'].nunique()

115

In [371]:
# Replace some duplicate vanues
game_df['venue'] = game_df['venue'].replace('Foxboro Stadium (Neutral Site)', "Foxboro Stadium")
game_df['venue'] = game_df['venue'].replace('Cotton Bowl', "Cotton Bowl, Dallas")
game_df['venue'] = game_df['venue'].replace('Sports Authority Field at Mile High, Denver', "Mile High Stadium")
game_df['venue'] = game_df['venue'].replace('Rose Bowl (Neutral Site)', "Rose Bowl")
game_df['venue'] = game_df['venue'].replace('MAPFRE Stadium', "MAPFRE Stadium, Columbus")
game_df['venue'] = game_df['venue'].replace('Robert F. Kennedy Memorial Stadium (Neutral Site)', "Robert F. Kennedy Memorial Stadium")
game_df['venue'] = game_df['venue'].replace('Giants Stadium', "Giants Stadium, East Rutherford")
game_df['venue'] = game_df['venue'].replace('Arrowhead Stadium', "Arrowhead Stadium, Kansas City")
game_df['venue'] = game_df['venue'].replace('Soldier Field', "Soldier Field, Chicago")
game_df['venue'] = game_df['venue'].replace(['Gillette Stadium', 'Gillette Stadium, Boston'], "Gillette Stadium, Foxborough")
game_df['venue'] = game_df['venue'].replace('Dignity Health Sports Park, Carson', "Dignity Health Sports Park, Los Angeles")
game_df['venue'] = game_df['venue'].replace(['RFK Stadium, Washington, D.C', 'RFK Memorial, Washington', 'R.F.K. Stadium, Washington'], "RFK Memorial Stadium, Washington, D.C.")
game_df['venue'] = game_df['venue'].replace("Dick's Sporting Goods Park, Denver", "Dick's Sporting Goods Park, Commerce City, CO")
game_df['venue'] = game_df['venue'].replace('BMO Field', "BMO Field, Toronto")
game_df['venue'] = game_df['venue'].replace('Buck Shaw Stadium', "Buck Shaw Stadium, California")
game_df['venue'] = game_df['venue'].replace(['Rio Tinto Stadium', 'Rio Tinto Stadium, Utah'], "Rio Tinto Stadium, Sandy, UT")
game_df['venue'] = game_df['venue'].replace(['Red Bull Arena', 'Red Bull Arena, Harrison'], "Red Bull Arena, New Jersey")
game_df['venue'] = game_df['venue'].replace('Subaru Park, Chester', "Subaru Park, Philadelphia")
game_df['venue'] = game_df['venue'].replace('Providence Park, Portland', "Providence Park, Portland, OR")
game_df['venue'] = game_df['venue'].replace("Children's Mercy Park, Kansas City, KS", "Children's Mercy Park, Kansas City")
game_df['venue'] = game_df['venue'].replace('Stanford Stadium', "Stanford, Palo Alto")
game_df['venue'] = game_df['venue'].replace(['Toyota Field', 'Toyota Stadium'], "Toyota Stadium, Frisco, TX")
game_df['venue'] = game_df['venue'].replace('BBVA Stadium', "BBVA Stadium, Houston, TX")
game_df['venue'] = game_df['venue'].replace('Yankee Stadium', "Yankee Stadium, New York")
game_df['venue'] = game_df['venue'].replace('Exploria Stadium', "Exploria Stadium, Orlando, Florida")
game_df['venue'] = game_df['venue'].replace('Mercedes-Benz Stadium', "Mercedes-Benz Stadium, Atlanta, Georgia")
game_df['venue'] = game_df['venue'].replace('Banc of California Stadium', "Banc of California Stadium, Los Angeles")
game_df['venue'] = game_df['venue'].replace('Audi Field', "Audi Field, Washington DC")
game_df['venue'] = game_df['venue'].replace('Allianz Field', "Allianz Field, Minnesota")
game_df['venue'] = game_df['venue'].replace('Nissan Stadium, Nashville', "Nissan Stadium, Nashville, Nashville, TN")

game_df['venue'].nunique()

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

80

In [372]:
game_df['venue'].unique()

array(['Foxboro Stadium', 'Cotton Bowl, Dallas', 'Mile High Stadium',
       'Rose Bowl', 'Robert F. Kennedy Memorial Stadium',
       "Houlihan's Stadium", 'Giants Stadium, East Rutherford',
       'Arrowhead Stadium, Kansas City', 'Ohio Stadium', 'CEFCU Stadium',
       'Soldier Field, Chicago', 'Lockhart Stadium',
       'Raymond James Stadium', 'MAPFRE Stadium, Columbus', nan,
       'AT THE ORANGE BOWL', 'Gillette Stadium, Foxborough',
       'Dignity Health Sports Park, Los Angeles',
       'Spartan Stadium, San Jose', 'RFK Stadium, Washington, D.C.',
       'Cardinal Stadium, Chicago', 'Dragon Stadium, Dallas',
       'Rice-Eccles Stadium, Salt Lake City',
       'Frisco Sports & Entertainment Center',
       'Los Angeles Memorial Coliseum, Los Angeles', 'Pizza Hut Park',
       'Robertson Stadium, Houston', 'Bridgeview Stadium',
       'SeatGeek Stadium', 'Reliant Stadium, Houston',
       "Dick's Sporting Goods Park, Commerce City, CO",
       'BMO Field, Toronto', 'Community 

In [373]:
game_df[game_df['attendance'].isnull()]

,home,away,date,year,time (utc),attendance,venue,Outcome
764,Tampa Bay,Columbus,3/18/2000,2000,NaN,NaN,Raymond James Stadium,win
765,LA Galaxy,Colorado,3/18/2000,2000,NaN,NaN,Rose Bowl,win
766,Miami,New England,3/18/2000,2000,NaN,NaN,Lockhart Stadium,tie
767,Dallas,Chicago,3/18/2000,2000,NaN,NaN,"Cotton Bowl, Dallas",win
768,Tampa Bay,New England,3/25/2000,2000,NaN,NaN,Raymond James Stadium,win
...,...,...,...,...,...,...,...,...
6669,Portland Timbers,Minnesota United FC,"Saturday, June 26",2021,3:30,NaN,"Providence Park, Portland, OR",loss
6677,CF Montréal,Inter Miami CF,"Saturday, July 3",2021,23:30,NaN,"Red Bull Arena, New Jersey",win
6687,New England Revolution,Toronto FC,"Wednesday, July 7",2021,23:00,NaN,"Gillette Stadium, Foxborough",loss
6688,CF Montréal,New York City FC,"Wednesday, July 7",2021,23:30,NaN,"Exploria Stadium, Orlando, Florida",win


In [374]:
audience = game_df['attendance']
stadium = game_df['venue']

null_stadium = []
for i in range(len(stadium)):
    if (audience[i] == 'NaN'):
        null_stadium.append(stadium[i])
null_stadium

[]

In [379]:
audience = game_df['attendance']
stadium = game_df['venue']

audience_rosebowl = []
for i in range(len(stadium)):
    if stadium[i] == 'Rose Bowl':
        audience_rosebowl.append(audience[i]) 

no_nan_rosebowl = [x for x in audience_rosebowl if pd.notnull(x)]

capacity_rosebowl = sum(no_nan_rosebowl)/len(no_nan_rosebowl)
capacity_rosebowl


22445.263157894737

In [381]:
len(no_nan_rosebowl)

76

In [ ]:
no_audience_stadium = [x for x in au]

In [382]:
audience_lockhart = []
for i in range(len(stadium)):
    if stadium[i] == 'Lockhart Stadium':
        audience_lockhart.append(audience[i]) 

        no_nan_lockhart = [x for x in audience_lockhart if pd.notnull(x)]

capacity_lockhart = sum(no_nan_lockhart)/len(no_nan_lockhart)
capacity_lockhart

9563.029411764706

In [351]:
for i in range(len(stadium)):
    if ((audience[i] == 'NaN') and (stadium[i] == 'Rose Bowl')):
        index[i]

In [385]:
for i in range(len(stadium)):
    if (pd.isnull(audience[i]) and (stadium[i] == 'Rose Bowl')):
        audience[i] = capacity_rosebowl
        #audience[i].fillna(capacity_foxboro, inplace = True)
audience.isnull().sum()

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3418: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


2339

In [386]:
for i in range(len(stadium)):
    if (pd.isnull(audience[i]) and (stadium[i] == 'Lockhart Stadium')):
        audience[i] = capacity_lockhart
        
audience.isnull().sum()

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


2323

In [387]:
audience_raymond = []
for i in range(len(stadium)):
    if stadium[i] == 'Raymond James Stadium':
        audience_raymond.append(audience[i]) 

        no_nan_raymond = [x for x in audience_raymond if pd.notnull(x)]

capacity_raymond = sum(no_nan_raymond)/len(no_nan_raymond)
capacity_raymond

12759.722222222223

In [388]:
for i in range(len(stadium)):
    if (pd.isnull(audience[i]) and (stadium[i] == 'Raymond James Stadium')):
        audience[i] = capacity_raymond
        
audience.isnull().sum()

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


2307

In [389]:
audience_cotton = []
for i in range(len(stadium)):
    if stadium[i] == 'Cotton Bowl, Dallas':
        audience_cotton.append(audience[i]) 

        no_nan_cotton = [x for x in audience_cotton if pd.notnull(x)]

capacity_cotton = sum(no_nan_cotton)/len(no_nan_cotton)
capacity_cotton

11923.205479452055

In [390]:
for i in range(len(stadium)):
    if (pd.isnull(audience[i]) and (stadium[i] == 'Cotton Bowl, Dallas')):
        audience[i] = capacity_cotton
        
audience.isnull().sum()

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


2271

In [392]:
audience_providence = []
for i in range(len(stadium)):
    if stadium[i] == 'Providence Park, Portland, OR':
        audience_providence.append(audience[i]) 

        no_nan_providence = [x for x in audience_providence if pd.notnull(x)]

capacity_providence = sum(no_nan_providence)/len(no_nan_providence)
capacity_providence

21503.88888888889

In [393]:
for i in range(len(stadium)):
    if (pd.isnull(audience[i]) and (stadium[i] == 'Providence Park, Portland, OR')):
        audience[i] = capacity_providence
        
audience.isnull().sum()

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


2231

In [394]:
audience_redbull = []
for i in range(len(stadium)):
    if stadium[i] == 'Red Bull Arena, New Jersey':
        audience_redbull.append(audience[i]) 

        no_nan_redbull = [x for x in audience_redbull if pd.notnull(x)]

capacity_redbull = sum(no_nan_redbull)/len(no_nan_redbull)
capacity_redbull

18930.718954248365

In [395]:
for i in range(len(stadium)):
    if (pd.isnull(audience[i]) and (stadium[i] == 'Red Bull Arena, New Jersey')):
        audience[i] = capacity_redbull
        
audience.isnull().sum()

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


2171

In [396]:
audience_gillette = []
for i in range(len(stadium)):
    if stadium[i] == 'Gillette Stadium, Foxborough':
        audience_gillette.append(audience[i]) 

        no_nan_gillette = [x for x in audience_gillette if pd.notnull(x)]

capacity_gillette = sum(no_nan_gillette)/len(no_nan_gillette)
capacity_gillette

16855.757396449702

In [397]:
for i in range(len(stadium)):
    if (pd.isnull(audience[i]) and (stadium[i] == 'Gillette Stadium, Foxborough')):
        audience[i] = capacity_gillette
        
audience.isnull().sum()

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


2046

In [398]:
audience_exploria = []
for i in range(len(stadium)):
    if stadium[i] == 'Exploria Stadium, Orlando, Florida':
        audience_exploria.append(audience[i]) 

        no_nan_exploria = [x for x in audience_exploria if pd.notnull(x)]

capacity_exploria = sum(no_nan_exploria)/len(no_nan_exploria)
capacity_exploria

22985.053571428572

In [399]:
for i in range(len(stadium)):
    if (pd.isnull(audience[i]) and (stadium[i] == 'Exploria Stadium, Orlando, Florida')):
        audience[i] = capacity_exploria
        
audience.isnull().sum()

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


2029

In [401]:
audience_dicks = []
for i in range(len(stadium)):
    if stadium[i] == "Dick's Sporting Goods Park, Commerce City, CO":
        audience_dicks.append(audience[i]) 

        no_nan_dicks = [x for x in audience_dicks if pd.notnull(x)]

capacity_dicks = sum(no_nan_dicks)/len(no_nan_dicks)
capacity_dicks

14995.89349112426

In [402]:
for i in range(len(stadium)):
    if (pd.isnull(audience[i]) and (stadium[i] == "Dick's Sporting Goods Park, Commerce City, CO")):
        audience[i] = capacity_dicks
        
audience.isnull().sum()

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


1967

In [403]:
# Check for new 'venue' that still has null in 'attendance'
game_df[game_df['attendance'].isnull()]

,home,away,date,year,time (utc),attendance,venue,Outcome
769,KC Wizards,Chicago,3/25/2000,2000,NaN,NaN,"Arrowhead Stadium, Kansas City",win
770,D.C. United,LA Galaxy,3/25/2000,2000,NaN,NaN,Robert F. Kennedy Memorial Stadium,loss
771,Columbus,San Jose,3/25/2000,2000,NaN,NaN,"MAPFRE Stadium, Columbus",win
775,KC Wizards,Colorado,4/1/2000,2000,NaN,NaN,"Arrowhead Stadium, Kansas City",win
776,MetroStars,D.C. United,4/1/2000,2000,NaN,NaN,"Giants Stadium, East Rutherford",win
...,...,...,...,...,...,...,...,...
6616,San Jose Earthquakes,Sporting Kansas City,"Saturday, May 22",2021,2:00,NaN,"PayPal Park, San Jose",loss
6627,LA Galaxy,San Jose Earthquakes,"Saturday, May 29",2021,23:00,NaN,"Dignity Health Sports Park, Los Angeles",win
6643,Austin FC,San Jose Earthquakes,"Saturday, June 19",2021,1:00,NaN,"Q2 Stadium, Austin",tie
6652,CF Montréal,DC United,"Wednesday, June 23",2021,0:00,NaN,"DRV PNK Stadium, Fort Lauderdale, FL",tie


In [404]:
audience_arrowhead = []
for i in range(len(stadium)):
    if stadium[i] == "Arrowhead Stadium, Kansas City":
        audience_arrowhead.append(audience[i]) 

        no_nan_arrowhead = [x for x in audience_arrowhead if pd.notnull(x)]

capacity_arrowhead = sum(no_nan_arrowhead)/len(no_nan_arrowhead)
capacity_arrowhead

9432.676056338029

In [405]:
for i in range(len(stadium)):
    if (pd.isnull(audience[i]) and (stadium[i] == "Arrowhead Stadium, Kansas City")):
        audience[i] = capacity_arrowhead
        
audience.isnull().sum()

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3418: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


1881

In [406]:
audience_rfk = []
for i in range(len(stadium)):
    if stadium[i] == "Robert F. Kennedy Memorial Stadium":
        audience_rfk.append(audience[i]) 

        no_nan_rfk = [x for x in audience_rfk if pd.notnull(x)]

capacity_rfk = sum(no_nan_rfk)/len(no_nan_rfk)
capacity_rfk

17415.25974025974

In [407]:
for i in range(len(stadium)):
    if (pd.isnull(audience[i]) and (stadium[i] == "Robert F. Kennedy Memorial Stadium")):
        audience[i] = capacity_rfk
        
audience.isnull().sum()

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


1865

In [408]:
audience_mapfre = []
for i in range(len(stadium)):
    if stadium[i] == "MAPFRE Stadium, Columbus":
        audience_mapfre.append(audience[i]) 

        no_nan_mapfre = [x for x in audience_mapfre if pd.notnull(x)]

capacity_mapfre = sum(no_nan_mapfre)/len(no_nan_mapfre)
capacity_mapfre

14992.549504950495

In [409]:
for i in range(len(stadium)):
    if (pd.isnull(audience[i]) and (stadium[i] == "MAPFRE Stadium, Columbus")):
        audience[i] = capacity_mapfre
        
audience.isnull().sum()

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


1743

In [410]:
audience_giants = []
for i in range(len(stadium)):
    if stadium[i] == "Giants Stadium, East Rutherford":
        audience_giants.append(audience[i]) 

        no_nan_giants = [x for x in audience_giants if pd.notnull(x)]

capacity_giants = sum(no_nan_giants)/len(no_nan_giants)
capacity_giants

16805.291666666668

In [411]:
for i in range(len(stadium)):
    if (pd.isnull(audience[i]) and (stadium[i] == "Giants Stadium, East Rutherford")):
        audience[i] = capacity_giants
        
audience.isnull().sum()

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


1655

In [ ]:
# Try to make a list of 'venue' where 'attendance' is NaN

In [425]:
venue_null_attendance = []
for i in range(len(stadium)):
    if pd.isnull(audience[i]) :
        venue_null_attendance.append(stadium[i]) 

set(venue_null_attendance) # Finding unique values from the 'list'

{'AT THE ORANGE BOWL',
 'Allianz Field, Minnesota',
 'Audi Field, Washington DC',
 'BBVA Stadium, Houston, TX',
 'BC Place, Vancouver',
 'BMO Field, Toronto',
 'Banc of California Stadium, Los Angeles',
 'Bridgeview Stadium',
 'Buck Shaw Stadium, California',
 'CEFCU Stadium',
 'Cardinal Stadium, Chicago',
 'CenturyLink Field, Seattle',
 "Children's Mercy Park, Kansas City",
 'Community America Ballpark',
 'DRV PNK Stadium, Fort Lauderdale, FL',
 'Dallas Cowboys New Stadium, Dallas',
 'Dignity Health Sports Park, Los Angeles',
 'Dragon Stadium, Dallas',
 'ESPN Wide World of Sports Complex',
 'Earthquakes Stadium',
 'Empire Field',
 'Foxboro Stadium',
 'Frisco Sports & Entertainment Center',
 'Inter Miami CF Stadium',
 'Los Angeles Memorial Coliseum, Los Angeles',
 'Mercedes-Benz Stadium, Atlanta, Georgia',
 'Mile High Stadium',
 'Nippert Stadium',
 'Nissan Stadium, Nashville, Nashville, TN',
 'PayPal Park, San Jose',
 'Pizza Hut Park',
 'Pratt & Whitney Stadium at Rentschler Field',
 '

In [426]:
len(venue_null_attendance)

1655

In [446]:
audience = game_df['attendance']
stadium = game_df['venue']

for i in range(len(venue_null_attendance)):
    audience_list = []
    
    for j in range(len(stadium)):
        if (stadium[j] in venue_null_attendance) :
            audience_list.append(audience[j]) 

            no_nan_list = [x for x in audience_list if pd.notnull(x)]
            capacity = sum(no_nan_list)/len(no_nan_list)
            
            if pd.isnull(audience[j]) :
                audience[j] = capacity
            
            

KeyboardInterrupt: 

In [452]:
game_df

,home,away,date,year,time (utc),attendance,venue,Outcome
0,New England,San Jose,7/31/1996,1996,NaN,12871.0,Foxboro Stadium,win
1,Dallas,Colorado,6/15/1996,1996,NaN,9704.0,"Cotton Bowl, Dallas",tie
2,Colorado,D.C. United,8/29/1996,1996,NaN,6368.0,Mile High Stadium,loss
3,LA Galaxy,New England,8/8/1996,1996,NaN,10251.0,Rose Bowl,win
4,New England,D.C. United,7/20/1996,1996,NaN,18347.0,Foxboro Stadium,win
...,...,...,...,...,...,...,...,...
6693,Vancouver Whitecaps,Real Salt Lake,"Wednesday, July 7",2021,2:00,12009.0,"Rio Tinto Stadium, Sandy, UT",loss
6694,LA Galaxy,FC Dallas,"Wednesday, July 7",2021,2:30,16453.0,"Dignity Health Sports Park, Los Angeles",win
6695,Nashville SC,Atlanta United FC,"Thursday, July 8",2021,0:30,22913.0,"Nissan Stadium, Nashville, Nashville, TN",tie
6696,New York Red Bulls,Philadelphia Union,"Thursday, July 8",2021,0:30,10013.0,"Red Bull Arena, New Jersey",tie


In [451]:
game_df.isnull().sum()


home            0
away            0
date            0
year            0
time (utc)    972
attendance      0
venue         467
Outcome         0
dtype: int64

In [455]:
game_df[771:790]

,home,away,date,year,time (utc),attendance,venue,Outcome
771,Columbus,San Jose,3/25/2000,2000,NaN,14992.549505,"MAPFRE Stadium, Columbus",win
772,Dallas,Colorado,3/25/2000,2000,NaN,11923.205479,"Cotton Bowl, Dallas",loss
773,Miami,MetroStars,3/26/2000,2000,NaN,9563.029412,Lockhart Stadium,win
774,Dallas,Tampa Bay,4/1/2000,2000,NaN,11923.205479,"Cotton Bowl, Dallas",win
775,KC Wizards,Colorado,4/1/2000,2000,NaN,9432.676056,"Arrowhead Stadium, Kansas City",win
776,MetroStars,D.C. United,4/1/2000,2000,NaN,16805.291667,"Giants Stadium, East Rutherford",win
777,LA Galaxy,New England,4/1/2000,2000,NaN,22445.263158,Rose Bowl,tie
778,Columbus,Chicago,4/1/2000,2000,NaN,14992.549505,"MAPFRE Stadium, Columbus",loss
779,Miami,San Jose,4/1/2000,2000,NaN,9563.029412,Lockhart Stadium,tie
780,Columbus,LA Galaxy,4/8/2000,2000,NaN,14992.549505,"MAPFRE Stadium, Columbus",tie


In [342]:
game_df.isnull().sum()

home             0
away             0
date             0
year             0
time (utc)     972
attendance    2355
venue          467
Outcome          0
dtype: int64

In [457]:
game_df[game_df['venue'].isnull()]

,home,away,date,year,time (utc),attendance,venue,Outcome
973,New York Red Bulls,New England Revolution,"Saturday, April 7",2001,23:00,15755.659751,NaN,win
974,Columbus Crew SC,Chicago Fire FC,"Saturday, April 7",2001,23:30,15755.659751,NaN,tie
975,DC United,Sporting Kansas City,"Saturday, April 7",2001,23:30,15755.659751,NaN,win
976,Miami,Colorado Rapids,"Saturday, April 7",2001,23:30,15755.659751,NaN,win
977,Tampa Bay,FC Dallas,"Saturday, April 7",2001,0:30,15755.659751,NaN,win
...,...,...,...,...,...,...,...,...
6284,San Jose Earthquakes,Minnesota United FC,"Saturday, August 1",2020,0:00,20484.423149,NaN,loss
6285,New York City FC,Portland Timbers,"Saturday, August 1",2020,2:30,20484.423149,NaN,loss
6286,Philadelphia Union,Portland Timbers,"Wednesday, August 5",2020,0:00,20484.423149,NaN,loss
6287,Orlando City SC,Minnesota United FC,"Thursday, August 6",2020,0:00,20484.423149,NaN,win


In [459]:
home_teams = game_df['home'].unique()
home_teams

array(['New England', 'Dallas', 'Colorado', 'LA Galaxy', 'D.C. United',
       'Tampa Bay', 'MetroStars', 'KC Wiz', 'Columbus', 'San Jose',
       'KC Wizards', 'Chicago', 'Miami', 'New York Red Bulls',
       'Columbus Crew SC', 'DC United', 'San Jose Earthquakes',
       'Sporting Kansas City', 'Chicago Fire FC', 'Colorado Rapids',
       'FC Dallas', 'New England Revolution', 'East All-Stars',
       'Chivas USA', 'Real Salt Lake', 'Houston Dynamo', 'Toronto FC',
       'Seattle Sounders FC', 'Philadelphia Union', 'Vancouver Whitecaps',
       'Portland Timbers', 'Montreal Impact', 'Orlando City SC',
       'New York City FC', 'Atlanta United FC', 'Minnesota United FC',
       'LAFC', 'FC Cincinnati', 'Houston Dynamo FC', 'Nashville SC',
       'Inter Miami CF', 'CF Montréal', 'Columbus Crew', 'Austin FC'],
      dtype=object)

In [469]:
key_teams = []
value_venues = []
home_teams_venues = {}
for i in range(len(stadium)) :
    for team in home_teams :
        if (game_df['home'][i] == team) and (not (game_df['home'][i]) in key_teams) and (not (stadium[i]) in value_venues) :
            key_teams.append(game_df['home'][i])
        #if not (stadium[i]) in value_venues:
            value_venues.append(stadium[i])
            home_teams_venues = dict(zip(key_teams, value_venues))
home_teams_venues   

{'New England': 'Foxboro Stadium',
 'Dallas': 'Cotton Bowl, Dallas',
 'Colorado': 'Mile High Stadium',
 'LA Galaxy': 'Rose Bowl',
 'D.C. United': 'Robert F. Kennedy Memorial Stadium',
 'Tampa Bay': "Houlihan's Stadium",
 'MetroStars': 'Giants Stadium, East Rutherford',
 'KC Wiz': 'Arrowhead Stadium, Kansas City',
 'Columbus': 'Ohio Stadium',
 'San Jose': 'CEFCU Stadium',
 'Chicago': 'Soldier Field, Chicago',
 'Miami': 'Lockhart Stadium',
 'New York Red Bulls': nan,
 'New England Revolution': 'Gillette Stadium, Foxborough',
 'San Jose Earthquakes': 'Spartan Stadium, San Jose',
 'Columbus Crew SC': 'MAPFRE Stadium, Columbus',
 'DC United': 'RFK Stadium, Washington, D.C.',
 'Chicago Fire FC': 'Cardinal Stadium, Chicago',
 'FC Dallas': 'Dragon Stadium, Dallas',
 'Sporting Kansas City': 'Dignity Health Sports Park, Los Angeles',
 'Real Salt Lake': 'Rice-Eccles Stadium, Salt Lake City',
 'Chivas USA': 'Los Angeles Memorial Coliseum, Los Angeles',
 'Houston Dynamo': 'Robertson Stadium, Housto

In [472]:
home_teams_venues.update({'New York Red Bulls': 'Red Bull Arena, New Jersey', 'Inter Miami CF': 'Inter Miami CF Stadium'})
home_teams_venues

{'New England': 'Foxboro Stadium',
 'Dallas': 'Cotton Bowl, Dallas',
 'Colorado': 'Mile High Stadium',
 'LA Galaxy': 'Rose Bowl',
 'D.C. United': 'Robert F. Kennedy Memorial Stadium',
 'Tampa Bay': "Houlihan's Stadium",
 'MetroStars': 'Giants Stadium, East Rutherford',
 'KC Wiz': 'Arrowhead Stadium, Kansas City',
 'Columbus': 'Ohio Stadium',
 'San Jose': 'CEFCU Stadium',
 'Chicago': 'Soldier Field, Chicago',
 'Miami': 'Lockhart Stadium',
 'New York Red Bulls': 'Red Bull Arena, New Jersey',
 'New England Revolution': 'Gillette Stadium, Foxborough',
 'San Jose Earthquakes': 'Spartan Stadium, San Jose',
 'Columbus Crew SC': 'MAPFRE Stadium, Columbus',
 'DC United': 'RFK Stadium, Washington, D.C.',
 'Chicago Fire FC': 'Cardinal Stadium, Chicago',
 'FC Dallas': 'Dragon Stadium, Dallas',
 'Sporting Kansas City': 'Dignity Health Sports Park, Los Angeles',
 'Real Salt Lake': 'Rice-Eccles Stadium, Salt Lake City',
 'Chivas USA': 'Los Angeles Memorial Coliseum, Los Angeles',
 'Houston Dynamo': '

In [475]:
game_df[game_df['home'] == 'Minnesota United FC'] 

,home,away,date,year,time (utc),attendance,venue,Outcome
5011,Minnesota United FC,Atlanta United FC,"Sunday, March 12",2017,21:00,35043.0,TCF Bank Stadium,loss
5033,Minnesota United FC,Real Salt Lake,"Saturday, April 1",2017,0:00,17728.0,TCF Bank Stadium,win
5070,Minnesota United FC,Colorado Rapids,"Sunday, April 23",2017,22:00,17491.0,TCF Bank Stadium,win
5077,Minnesota United FC,San Jose Earthquakes,"Saturday, April 29",2017,0:00,17605.0,TCF Bank Stadium,loss
5093,Minnesota United FC,Sporting Kansas City,"Sunday, May 7",2017,17:30,17709.0,TCF Bank Stadium,win
...,...,...,...,...,...,...,...,...
6570,Minnesota United FC,Austin FC,"Saturday, May 1",2021,0:00,4100.0,"Allianz Field, Minnesota",loss
6592,Minnesota United FC,Vancouver Whitecaps,"Wednesday, May 12",2021,1:00,4100.0,"Allianz Field, Minnesota",win
6599,Minnesota United FC,FC Dallas,"Saturday, May 15",2021,0:00,4100.0,"Allianz Field, Minnesota",win
6654,Minnesota United FC,Austin FC,"Wednesday, June 23",2021,0:00,19209.0,"Allianz Field, Minnesota",win


In [485]:
for team in home_teams_venues:
    for i in range(len(stadium)) :
        if pd.isnull(stadium[i]) and (game_df['home'][i] == team) :
            stadium[i] = team
            

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3418: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [487]:
game_df.isnull().sum()

home            0
away            0
date            0
year            0
time (utc)    972
attendance      0
venue           0
Outcome         0
dtype: int64

In [489]:
game_df.dtypes

home           object
away           object
date           object
year            int64
time (utc)     object
attendance    float64
venue          object
Outcome        object
dtype: object

In [490]:
game_df.nunique()

home            44
away            44
date          1503
year            26
time (utc)      54
attendance    3557
venue          100
Outcome          3
dtype: int64

In [496]:
game_df['date'] = pd.to_datetime(game_df['date'], errors = 'coerce')
#game_df = game_df.drop(["time (utc)"], axis='columns')

game_df.head(2)

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,home,away,date,year,time (utc),attendance,venue,Outcome
0,New England,San Jose,1996-07-31,1996,NaN,12871.0,Foxboro Stadium,win
1,Dallas,Colorado,1996-06-15,1996,NaN,9704.0,"Cotton Bowl, Dallas",tie


In [497]:
game_df.dtypes

home                  object
away                  object
date          datetime64[ns]
year                   int64
time (utc)            object
attendance           float64
venue                 object
Outcome               object
dtype: object

In [498]:
game_df.nunique()

home            44
away            44
date           461
year            26
time (utc)      54
attendance    3557
venue          100
Outcome          3
dtype: int64

In [499]:
game_df['away'].unique()

array(['San Jose', 'Colorado', 'D.C. United', 'New England', 'Columbus',
       'Dallas', 'KC Wiz', 'Tampa Bay', 'MetroStars', 'LA Galaxy',
       'KC Wizards', 'Miami', 'Chicago', 'New England Revolution',
       'Chicago Fire FC', 'Sporting Kansas City', 'Colorado Rapids',
       'FC Dallas', 'DC United', 'New York Red Bulls',
       'San Jose Earthquakes', 'Columbus Crew SC', 'West All-Stars',
       'Real Salt Lake', 'Chivas USA', 'Houston Dynamo', 'Toronto FC',
       'Seattle Sounders FC', 'Philadelphia Union', 'Portland Timbers',
       'Vancouver Whitecaps', 'Montreal Impact', 'New York City FC',
       'Orlando City SC', 'Minnesota United FC', 'Atlanta United FC',
       'LAFC', 'FC Cincinnati', 'Inter Miami CF', 'Houston Dynamo FC',
       'Nashville SC', 'Austin FC', 'CF Montréal', 'Columbus Crew'],
      dtype=object)

In [500]:
game_df['home'] = game_df['home'].replace('Colorado', "Colorado Rapids")
game_df['away'] = game_df['away'].replace('Colorado', "Colorado Rapids")

game_df['home'] = game_df['home'].replace('Columbus Crew SC', "Columbus Crew")
game_df['away'] = game_df['away'].replace('Columbus Crew SC', "Columbus Crews")

game_df['home'] = game_df['home'].replace('Houston Dynamo', "Houston Dynamo FC")
game_df['away'] = game_df['away'].replace('Houston Dynamo', "Houston Dynamo FC")

game_df.head()

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

,home,away,date,year,time (utc),attendance,venue,Outcome
0,New England,San Jose,1996-07-31,1996,NaN,12871.0,Foxboro Stadium,win
1,Dallas,Colorado Rapids,1996-06-15,1996,NaN,9704.0,"Cotton Bowl, Dallas",tie
2,Colorado Rapids,D.C. United,1996-08-29,1996,NaN,6368.0,Mile High Stadium,loss
3,LA Galaxy,New England,1996-08-08,1996,NaN,10251.0,Rose Bowl,win
4,New England,D.C. United,1996-07-20,1996,NaN,18347.0,Foxboro Stadium,win


In [501]:
game_df['home'].unique()

array(['New England', 'Dallas', 'Colorado Rapids', 'LA Galaxy',
       'D.C. United', 'Tampa Bay', 'MetroStars', 'KC Wiz', 'Columbus',
       'San Jose', 'KC Wizards', 'Chicago', 'Miami', 'New York Red Bulls',
       'Columbus Crew', 'DC United', 'San Jose Earthquakes',
       'Sporting Kansas City', 'Chicago Fire FC', 'FC Dallas',
       'New England Revolution', 'East All-Stars', 'Chivas USA',
       'Real Salt Lake', 'Houston Dynamo FC', 'Toronto FC',
       'Seattle Sounders FC', 'Philadelphia Union', 'Vancouver Whitecaps',
       'Portland Timbers', 'Montreal Impact', 'Orlando City SC',
       'New York City FC', 'Atlanta United FC', 'Minnesota United FC',
       'LAFC', 'FC Cincinnati', 'Nashville SC', 'Inter Miami CF',
       'CF Montréal', 'Austin FC'], dtype=object)

In [502]:
# Check unique entries
game_df['time (utc)'].unique()

array([nan, '13:00 (14:00)', '23:00', '23:30', '0:30', '2:00', '1:00',
       '19:00', '20:00', '0:00', '18:00', '17:00', '2:30', '22:00',
       '3:00', '19:30', '16:30', '21:00', '1:30', '20:30', '22:30',
       '7:00', '17:30', '16:00', '8:00', '0:20', '21:30', '13:00', '3:30',
       '4:25', '15:00', '18:30', '4:00', '21:55', '0:10', '1:44', '2:15',
       '1:15', '3:35', '19:55', '17:55', '0:55', '1:27', '2:40', '20:55',
       '1:47', '0:15', '1:38', '1:36', '0:45', '3:20', '1:45', '1:52',
       '23:15', '0:02'], dtype=object)

In [503]:
# Check 'time' column
game_df['time (utc)'] = game_df['time (utc)'].replace('13:00 (14:00)', "14:00")
#game_df['time (utc)'].drop(game_df['time (utc)'] == '13:00 (14:00)')
game_df['time (utc)'] 

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
        ...  
6693     2:00
6694     2:30
6695     0:30
6696     0:30
6697    23:30
Name: time (utc), Length: 6698, dtype: object

In [504]:
game_df['date'].unique

<bound method Series.unique of 0      1996-07-31
1      1996-06-15
2      1996-08-29
3      1996-08-08
4      1996-07-20
          ...    
6693          NaT
6694          NaT
6695          NaT
6696          NaT
6697          NaT
Name: date, Length: 6698, dtype: datetime64[ns]>

In [506]:
game_df = game_df.drop(["date", "time (utc)"], axis='columns')
game_df.head(2)

,home,away,year,attendance,venue,Outcome
0,New England,San Jose,1996,12871.0,Foxboro Stadium,win
1,Dallas,Colorado Rapids,1996,9704.0,"Cotton Bowl, Dallas",tie


In [507]:
game_df.isnull().sum()


home          0
away          0
year          0
attendance    0
venue         0
Outcome       0
dtype: int64

In [508]:
game_df.nunique()

home            41
away            42
year            26
attendance    3557
venue          100
Outcome          3
dtype: int64

In [509]:
game_df.dtypes

home           object
away           object
year            int64
attendance    float64
venue          object
Outcome        object
dtype: object

It looks like there are 4 columns with **categorical** values having consistent data type as *object*.

We should **generate a list of categorical variable** names using Python's "df.dtypes" property. In that case, we can use our variable list to perform the **one-hot encoding** *once*, rather than for each individual variable.

In [510]:
# Generate our categorical variable list
game_cat = game_df.dtypes[game_df.dtypes == "object"].index.tolist()
game_cat

['home', 'away', 'venue', 'Outcome']

In [511]:
# Check the number of unique values in each column
game_df[game_cat].nunique()

home        41
away        42
venue      100
Outcome      3
dtype: int64

In [512]:
game_df['home'].unique()

array(['New England', 'Dallas', 'Colorado Rapids', 'LA Galaxy',
       'D.C. United', 'Tampa Bay', 'MetroStars', 'KC Wiz', 'Columbus',
       'San Jose', 'KC Wizards', 'Chicago', 'Miami', 'New York Red Bulls',
       'Columbus Crew', 'DC United', 'San Jose Earthquakes',
       'Sporting Kansas City', 'Chicago Fire FC', 'FC Dallas',
       'New England Revolution', 'East All-Stars', 'Chivas USA',
       'Real Salt Lake', 'Houston Dynamo FC', 'Toronto FC',
       'Seattle Sounders FC', 'Philadelphia Union', 'Vancouver Whitecaps',
       'Portland Timbers', 'Montreal Impact', 'Orlando City SC',
       'New York City FC', 'Atlanta United FC', 'Minnesota United FC',
       'LAFC', 'FC Cincinnati', 'Nashville SC', 'Inter Miami CF',
       'CF Montréal', 'Austin FC'], dtype=object)

Before going to encode these columns using Scikit-learn's OneHotEncoder module, we need to make sure that if the categorical variables require **bucketing**. Since we want to predict outcome for each individual `home` team, we need to see the relation of all individual `home` team, `venue` and `game_status`, we choose not to to any binning/bucketing, and we're ready to use **OneHotEncoder**.

In [513]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(game_df[game_cat]))
encode_df.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [514]:
# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(game_cat)
encode_df.head()

,home_Atlanta United FC,home_Austin FC,home_CF Montréal,home_Chicago,home_Chicago Fire FC,home_Chivas USA,home_Colorado Rapids,home_Columbus,home_Columbus Crew,home_D.C. United,home_DC United,home_Dallas,home_East All-Stars,home_FC Cincinnati,home_FC Dallas,home_Houston Dynamo FC,home_Inter Miami CF,home_KC Wiz,home_KC Wizards,home_LA Galaxy,home_LAFC,home_MetroStars,home_Miami,home_Minnesota United FC,home_Montreal Impact,home_Nashville SC,home_New England,home_New England Revolution,home_New York City FC,home_New York Red Bulls,home_Orlando City SC,home_Philadelphia Union,home_Portland Timbers,home_Real Salt Lake,home_San Jose,home_San Jose Earthquakes,home_Seattle Sounders FC,home_Sporting Kansas City,home_Tampa Bay,home_Toronto FC,home_Vancouver Whitecaps,away_Atlanta United FC,away_Austin FC,away_CF Montréal,away_Chicago,away_Chicago Fire FC,away_Chivas USA,away_Colorado Rapids,away_Columbus,away_Columbus Crew,away_Columbus Crews,away_D.C. United,away_DC United,away_Dallas,away_FC Cincinnati,away_FC Dallas,away_Houston Dynamo FC,away_Inter Miami CF,away_KC Wiz,away_KC Wizards,away_LA Galaxy,away_LAFC,away_MetroStars,away_Miami,away_Minnesota United FC,away_Montreal Impact,away_Nashville SC,away_New England,away_New England Revolution,away_New York City FC,away_New York Red Bulls,away_Orlando City SC,away_Philadelphia Union,away_Portland Timbers,away_Real Salt Lake,away_San Jose,away_San Jose Earthquakes,away_Seattle Sounders FC,away_Sporting Kansas City,away_Tampa Bay,away_Toronto FC,away_Vancouver Whitecaps,away_West All-Stars,venue_AT THE ORANGE BOWL,"venue_Allianz Field, Minnesota","venue_Arrowhead Stadium, Kansas City","venue_Audi Field, Washington DC","venue_BBVA Stadium, Houston, TX","venue_BC Place, Vancouver","venue_BMO Field, Toronto","venue_Banc of California Stadium, Los Angeles",venue_Bobby Dodd Stadium,venue_Bridgeview Stadium,"venue_Buck Shaw Stadium, California",venue_CEFCU Stadium,"venue_Camping World Stadium, Orlando","venue_Cardinal Stadium, Chicago","venue_CenturyLink Field, Seattle",venue_Chicago Fire FC,"venue_Children's Mercy Park, Kansas City",venue_Chivas USA,venue_Citi Field Stadium,venue_Colorado Rapids,venue_Columbus Crew SC,venue_Community America Ballpark,"venue_Cotton Bowl, Dallas",venue_DC United,"venue_DRV PNK Stadium, Fort Lauderdale, FL","venue_Dallas Cowboys New Stadium, Dallas","venue_Dick's Sporting Goods Park, Commerce City, CO","venue_Dignity Health Sports Park, Los Angeles","venue_Dragon Stadium, Dallas",venue_ESPN Wide World of Sports Complex,venue_Earthquakes Stadium,venue_Empire Field,"venue_Exploria Stadium, Orlando, Florida",venue_FC Dallas,venue_FedExField,venue_Foxboro Stadium,venue_Frisco Sports & Entertainment Center,"venue_Giants Stadium, East Rutherford","venue_Gillette Stadium, Foxborough","venue_Historic Crew Stadium, Columbus, OH",venue_Houlihan's Stadium,venue_Inter Miami CF Stadium,venue_LA Galaxy,venue_Levi's Stadium,venue_Lockhart Stadium,"venue_Los Angeles Memorial Coliseum, Los Angeles","venue_Lower.com Field, Columbus, OH","venue_Lumen Field, Seattle","venue_MAPFRE Stadium, Columbus","venue_Maryland SoccerPlex, Germantown",venue_McAfee Coliseum,"venue_Mercedes-Benz Stadium, Atlanta, Georgia",venue_Miami,venue_Mile High Stadium,venue_Navy-Marine Corps Memorial Stadium,venue_New England Revolution,venue_New York City FC,venue_New York Red Bulls,venue_Nippert Stadium,"venue_Nissan Stadium, Nashville, Nashville, TN",venue_Ohio Stadium,venue_Orlando City SC,"venue_PayPal Park, San Jose",venue_Philadelphia Union,venue_Pizza Hut Park,venue_Portland Timbers,venue_Pratt & Whitney Stadium at Rentschler Field,"venue_Providence Park, Portland, OR","venue_Q2 Stadium, Austin","venue_Qwest Field, Seattle","venue_RFK Memorial Stadium, Washington, D.C.","venue_RFK Stadium, Washington, D.C.",venue_Raymond James Stadium,venue_Real Salt Lake,"venue_Red Bull Arena, New Jersey","venue_Reliant Stadium, Houston","venue_Rice-Eccles Stadium, Salt Lake City","venue_Rio Tinto Stadium, Sand

Now that our categorical variables have been encoded, they are ready to replace our unencoded categorical variables in our dataset using *panda's* **merge** and **drop** methods.

In [515]:
# Merge OneHotEncoded features and drop the originals from the updated df
game_df = game_df.merge(encode_df, left_index=True, right_index=True).drop(game_df[game_cat],1)
game_df.head()

,year,attendance,home_Atlanta United FC,home_Austin FC,home_CF Montréal,home_Chicago,home_Chicago Fire FC,home_Chivas USA,home_Colorado Rapids,home_Columbus,home_Columbus Crew,home_D.C. United,home_DC United,home_Dallas,home_East All-Stars,home_FC Cincinnati,home_FC Dallas,home_Houston Dynamo FC,home_Inter Miami CF,home_KC Wiz,home_KC Wizards,home_LA Galaxy,home_LAFC,home_MetroStars,home_Miami,home_Minnesota United FC,home_Montreal Impact,home_Nashville SC,home_New England,home_New England Revolution,home_New York City FC,home_New York Red Bulls,home_Orlando City SC,home_Philadelphia Union,home_Portland Timbers,home_Real Salt Lake,home_San Jose,home_San Jose Earthquakes,home_Seattle Sounders FC,home_Sporting Kansas City,home_Tampa Bay,home_Toronto FC,home_Vancouver Whitecaps,away_Atlanta United FC,away_Austin FC,away_CF Montréal,away_Chicago,away_Chicago Fire FC,away_Chivas USA,away_Colorado Rapids,away_Columbus,away_Columbus Crew,away_Columbus Crews,away_D.C. United,away_DC United,away_Dallas,away_FC Cincinnati,away_FC Dallas,away_Houston Dynamo FC,away_Inter Miami CF,away_KC Wiz,away_KC Wizards,away_LA Galaxy,away_LAFC,away_MetroStars,away_Miami,away_Minnesota United FC,away_Montreal Impact,away_Nashville SC,away_New England,away_New England Revolution,away_New York City FC,away_New York Red Bulls,away_Orlando City SC,away_Philadelphia Union,away_Portland Timbers,away_Real Salt Lake,away_San Jose,away_San Jose Earthquakes,away_Seattle Sounders FC,away_Sporting Kansas City,away_Tampa Bay,away_Toronto FC,away_Vancouver Whitecaps,away_West All-Stars,venue_AT THE ORANGE BOWL,"venue_Allianz Field, Minnesota","venue_Arrowhead Stadium, Kansas City","venue_Audi Field, Washington DC","venue_BBVA Stadium, Houston, TX","venue_BC Place, Vancouver","venue_BMO Field, Toronto","venue_Banc of California Stadium, Los Angeles",venue_Bobby Dodd Stadium,venue_Bridgeview Stadium,"venue_Buck Shaw Stadium, California",venue_CEFCU Stadium,"venue_Camping World Stadium, Orlando","venue_Cardinal Stadium, Chicago","venue_CenturyLink Field, Seattle",venue_Chicago Fire FC,"venue_Children's Mercy Park, Kansas City",venue_Chivas USA,venue_Citi Field Stadium,venue_Colorado Rapids,venue_Columbus Crew SC,venue_Community America Ballpark,"venue_Cotton Bowl, Dallas",venue_DC United,"venue_DRV PNK Stadium, Fort Lauderdale, FL","venue_Dallas Cowboys New Stadium, Dallas","venue_Dick's Sporting Goods Park, Commerce City, CO","venue_Dignity Health Sports Park, Los Angeles","venue_Dragon Stadium, Dallas",venue_ESPN Wide World of Sports Complex,venue_Earthquakes Stadium,venue_Empire Field,"venue_Exploria Stadium, Orlando, Florida",venue_FC Dallas,venue_FedExField,venue_Foxboro Stadium,venue_Frisco Sports & Entertainment Center,"venue_Giants Stadium, East Rutherford","venue_Gillette Stadium, Foxborough","venue_Historic Crew Stadium, Columbus, OH",venue_Houlihan's Stadium,venue_Inter Miami CF Stadium,venue_LA Galaxy,venue_Levi's Stadium,venue_Lockhart Stadium,"venue_Los Angeles Memorial Coliseum, Los Angeles","venue_Lower.com Field, Columbus, OH","venue_Lumen Field, Seattle","venue_MAPFRE Stadium, Columbus","venue_Maryland SoccerPlex, Germantown",venue_McAfee Coliseum,"venue_Mercedes-Benz Stadium, Atlanta, Georgia",venue_Miami,venue_Mile High Stadium,venue_Navy-Marine Corps Memorial Stadium,venue_New England Revolution,venue_New York City FC,venue_New York Red Bulls,venue_Nippert Stadium,"venue_Nissan Stadium, Nashville, Nashville, TN",venue_Ohio Stadium,venue_Orlando City SC,"venue_PayPal Park, San Jose",venue_Philadelphia Union,venue_Pizza Hut Park,venue_Portland Timbers,venue_Pratt & Whitney Stadium at Rentschler Field,"venue_Providence Park, Portland, OR","venue_Q2 Stadium, Austin","venue_Qwest Field, Seattle","venue_RFK Memorial Stadium, Washington, D.C.","venue_RFK Stadium, Washington, D.C.",venue_Raymond James Stadium,venue_Real Salt Lake,"venue_Red Bull Arena, New Jersey","venue_Reliant Stadium, Houston","venue_Rice-Eccles Stadium, Salt Lake City","venue_Rio Tin

We need to **split** our **training** and **testing** data *before* fitting our **StandardScaler** instance. This <u> prevents testing data from influencing the standardization </u> function.

To build our training and testing datasets, we need to separate two values:

input values (which are our *independent variables* commonly referred to as **model features or "X"**) and **target output** ( *dependent variable* commonly referred to as **target or "y"** in TensorFlow documentation).

We want to build a model that will predict whether or not a team is winning; therefore, we must separate the `Outcome_win` column from the rest of the input data. Also, the `Outcome_loss`, `Outcome_tie`columns have the same input for all the data, so we can drop this column.

In [527]:
# Split our preprocessed data into our features and target arrays
y = game_df["Outcome_win"].values
X = game_df.drop(["Outcome_win","Outcome_loss", "Outcome_tie" ], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

Now that our training and testing data have been allocated, we're ready to build our **StandardScalerobject** and standardize the numerical features.

In [528]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Our data is now **preprocessed** via one-hot encoding and standardization. 

### Define Neural Network model

For our **input layer**, we must add the **number of input features equal to the number of variables in our feature** DataFrame.

In our **hidden layers**, we'll add **three hidden layers** with only a few neurons in each layer. To create the *second hidden layer*, we'll add another **Keras Dense class** while defining our model. All of our hidden layers will use the **relu activation** function to identify nonlinear characteristics from the input values.

In the **output layer**, we'll use the `sigmoid` activation function that will help us predict the probability that a team is winning or not.

In [529]:
len(X_train[0])

185

In [530]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  370
hidden_nodes_layer2 = 185
hidden_nodes_layer3 = 100

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 370)               68820     
_________________________________________________________________
dense_25 (Dense)             (None, 185)               68635     
_________________________________________________________________
dense_26 (Dense)             (None, 100)               18600     
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 101       
Total params: 156,156
Trainable params: 156,156
Non-trainable params: 0
_________________________________________________________________


In [531]:
# Compile the model
#nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
157/157 [==============================] - 2s 7ms/step - loss: 0.2892 - accuracy: 0.5134
Epoch 2/100
157/157 [==============================] - 1s 6ms/step - loss: 0.2493 - accuracy: 0.5652
Epoch 3/100
157/157 [==============================] - 1s 8ms/step - loss: 0.2379 - accuracy: 0.5941
Epoch 4/100
157/157 [==============================] - 1s 9ms/step - loss: 0.2423 - accuracy: 0.5871
Epoch 5/100
157/157 [==============================] - 1s 9ms/step - loss: 0.2288 - accuracy: 0.6197
Epoch 6/100
157/157 [==============================] - 1s 8ms/step - loss: 0.2276 - accuracy: 0.6223
Epoch 7/100
157/157 [==============================] - 1s 8ms/step - loss: 0.2226 - accuracy: 0.6319
Epoch 8/100
157/157 [==============================] - 1s 9ms/step - loss: 0.2208 - accuracy: 0.6355
Epoch 9/100
157/157 [==============================] - 1s 9ms/step - loss: 0.2155 - accuracy: 0.6482
Epoch 10/100
157/157 [==============================] - 1s 9ms/step - loss: 0.2162 - accura

157/157 [==============================] - 2s 14ms/step - loss: 0.1066 - accuracy: 0.8477
Epoch 77/100
157/157 [==============================] - 2s 15ms/step - loss: 0.1017 - accuracy: 0.8531
Epoch 78/100
157/157 [==============================] - 2s 14ms/step - loss: 0.1043 - accuracy: 0.8445
Epoch 79/100
157/157 [==============================] - 2s 14ms/step - loss: 0.0997 - accuracy: 0.8495 0s - loss: 0.0981 - accu
Epoch 80/100
157/157 [==============================] - 2s 15ms/step - loss: 0.0993 - accuracy: 0.8515
Epoch 81/100
157/157 [==============================] - 2s 15ms/step - loss: 0.0976 - accuracy: 0.8567
Epoch 82/100
157/157 [==============================] - 2s 14ms/step - loss: 0.0994 - accuracy: 0.8525
Epoch 83/100
157/157 [==============================] - 2s 15ms/step - loss: 0.1008 - accuracy: 0.8533 0s - loss: 0.0 - ETA: 0s - loss: 0.1008 - accuracy: 
Epoch 84/100
157/157 [==============================] - 3s 18ms/step - loss: 0.0970 - accuracy: 0.8588 0s - los

In [ ]:
gaussian